In [29]:
from bs4 import BeautifulSoup
import requests
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [9]:
my_url = 'http://www.nytimes.com'

In [10]:
r = requests.get(my_url)

In [13]:
r.text

u'<!DOCTYPE html>\n<!--[if (gt IE 9)|!(IE)]> <!--> <html lang="en" class="no-js edition-domestic app-homepage"  itemscope xmlns:og="http://opengraphprotocol.org/schema/"> <!--<![endif]-->\n<!--[if IE 9]> <html lang="en" class="no-js ie9 lt-ie10 edition-domestic app-homepage" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->\n<!--[if IE 8]> <html lang="en" class="no-js ie8 lt-ie10 lt-ie9 edition-domestic app-homepage" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->\n<!--[if (lt IE 8)]> <html lang="en" class="no-js lt-ie10 lt-ie9 lt-ie8 edition-domestic app-homepage" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->\n<head>\n    <title>The New York Times - Breaking News, World News & Multimedia</title>\n        <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n<link rel="shortcut icon" href="https://static.nytimes.com/favicon.ico" />\n<link rel="apple-touch-icon-precomposed" sizes="144\xd7144" href="https://static.nytimes.com/images/i

In [43]:
soup = BeautifulSoup(r.text, "html.parser")

In [44]:
print soup.prettify()

<!DOCTYPE html>
<!--[if (gt IE 9)|!(IE)]> <!-->
<html class="no-js edition-domestic app-homepage" itemscope="" lang="en" xmlns:og="http://opengraphprotocol.org/schema/">
 <!--<![endif]-->
 <!--[if IE 9]> <html lang="en" class="no-js ie9 lt-ie10 edition-domestic app-homepage" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->
 <!--[if IE 8]> <html lang="en" class="no-js ie8 lt-ie10 lt-ie9 edition-domestic app-homepage" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->
 <!--[if (lt IE 8)]> <html lang="en" class="no-js lt-ie10 lt-ie9 lt-ie8 edition-domestic app-homepage" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->
 <head>
  <title>
   The New York Times - Breaking News, World News &amp; Multimedia
  </title>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <link href="https://static.nytimes.com/favicon.ico" rel="shortcut icon"/>
  <link href="https://static.nytimes.com/images/icons/ios-ipad-144x144.png" rel="apple-touch-icon-prec

In [50]:
paragraphs = soup.find_all("p", class_="summary")

In [52]:
stories_to_read = []
for item in paragraphs:
    stories_to_read.append(item.get_text())

In [53]:
print(stories_to_read)

[u'Gavin Newsom, a Democratic former mayor of San Francisco, and John Cox, a Republican backed by President Trump, won the two spots in the race to succeed Gov. Jerry Brown.\nBut the results were more muddled in the most-watched races in the state: seven congressional districts that Democrats are aiming to capture this November.', u'While the national focus remained largely on California, races in New Jersey, Iowa and other states were also crucial.', u'Facebook has data-sharing partnerships with at least four Chinese companies, including the manufacturing giant Huawei.\n\nHuawei has been flagged by American intelligence as a national security threat. It has a close relationship with China\u2019s government.', u'\n        President Trump acknowledged perhaps as explicitly as ever that he was angry at Attorney General Jeff Sessions for not stopping the Russia inquiry. To critics, it is all but an admission of guilt.    ', u'Mexico said it would impose tariffs on cheese, whiskey and othe

In [54]:
sentiments_list = []
for story in stories_to_read:
    analysis_subject = TextBlob(story, analyzer = NaiveBayesAnalyzer())
    sentiments_list.append(opinion.sentiment)

In [55]:
print sentiments_list

[Sentiment(classification='neg', p_pos=0.47979797979797983, p_neg=0.5202020202020203), Sentiment(classification='neg', p_pos=0.47979797979797983, p_neg=0.5202020202020203), Sentiment(classification='neg', p_pos=0.47979797979797983, p_neg=0.5202020202020203), Sentiment(classification='neg', p_pos=0.47979797979797983, p_neg=0.5202020202020203), Sentiment(classification='neg', p_pos=0.47979797979797983, p_neg=0.5202020202020203), Sentiment(classification='neg', p_pos=0.47979797979797983, p_neg=0.5202020202020203), Sentiment(classification='neg', p_pos=0.47979797979797983, p_neg=0.5202020202020203), Sentiment(classification='neg', p_pos=0.47979797979797983, p_neg=0.5202020202020203), Sentiment(classification='neg', p_pos=0.47979797979797983, p_neg=0.5202020202020203), Sentiment(classification='neg', p_pos=0.47979797979797983, p_neg=0.5202020202020203), Sentiment(classification='neg', p_pos=0.47979797979797983, p_neg=0.5202020202020203), Sentiment(classification='neg', p_pos=0.4797979797979